# **Project Name**    - Local Food Wastage Management System



##### **Project Type**    - EDA
##### **Contribution**    - Individual
##### **Team Member 1 -** Soham Mukherjee

# Import Libraries

In [24]:
import pandas as pd
from sqlalchemy import create_engine

#  Database

In [25]:
# A function to set up the SQLite database from your CSV files
def setup_database():
    """
    Loads CSV data into a SQLite database.
    """
    engine = create_engine('sqlite:///food_wastage.db')

    # Load the cleaned data from your CSV files
    providers_df = pd.read_csv('providers_cleaned.csv')
    receivers_df = pd.read_csv('receivers_cleaned.csv')
    food_listings_df = pd.read_csv('food_listings_cleaned.csv')
    claims_df = pd.read_csv('claims_cleaned.csv')

    # Write the dataframes to SQL tables
    providers_df.to_sql('providers', engine, index=False, if_exists='replace')
    receivers_df.to_sql('receivers', engine, index=False, if_exists='replace')
    food_listings_df.to_sql('food_listings', engine, index=False, if_exists='replace')
    claims_df.to_sql('claims', engine, index=False, if_exists='replace')
    
    return engine

# Connect to the database
engine = setup_database()
print("Database 'food_wastage.db' created successfully.")


Database 'food_wastage.db' created successfully.


# Questions to be Answered (SQL Queries & Analysis)

### 1. How many food providers and receivers are there in each city?

In [26]:

print("\n--- Providers by City ---")
providers_by_city_query = "SELECT City, COUNT(*) AS TotalProviders FROM providers GROUP BY City ORDER BY TotalProviders DESC;"
providers_by_city_df = pd.read_sql(providers_by_city_query, engine)
print(providers_by_city_df)

print("\n--- Receivers by City ---")
receivers_by_city_query = "SELECT City, COUNT(*) AS TotalReceivers FROM receivers GROUP BY City ORDER BY TotalReceivers DESC;"
receivers_by_city_df = pd.read_sql(receivers_by_city_query, engine)
print(receivers_by_city_df)


--- Providers by City ---
                         City  TotalProviders
0    South Christopherborough               3
1                   New Carol               3
2                 Williamview               2
3           West Lauraborough               2
4            West Christopher               2
..                        ...             ...
958          Alexanderchester               1
959              Aguirreville               1
960                Adamsville               1
961                 Adamsview               1
962                  Adambury               1

[963 rows x 2 columns]

--- Receivers by City ---
                City  TotalReceivers
0    New Christopher               3
1       Williamsfort               2
2       West Richard               2
3         West James               2
4        Stevenmouth               2
..               ...             ...
961    Alexanderbury               1
962      Aguilarstad               1
963      Aguilarbury               1


### 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

In [27]:

print("\n--- Most Contributing Provider Type ---")
most_contributing_provider_query = """
    SELECT p.Type, SUM(fl.Quantity) AS TotalQuantity
    FROM providers AS p
    JOIN food_listings AS fl
    ON p.Provider_ID = fl.Provider_ID
    GROUP BY p.Type
    ORDER BY TotalQuantity DESC;
"""
most_contributing_df = pd.read_sql(most_contributing_provider_query, engine)
print(most_contributing_df)


--- Most Contributing Provider Type ---
               Type  TotalQuantity
0        Restaurant           6923
1       Supermarket           6696
2  Catering Service           6116
3     Grocery Store           6059


### 3. What is the contact information of food providers in a specific city?

In [28]:

# (Replace 'New Jessica' with the city you want to query)
print("\n--- Contact Information for Providers in 'New Jessica' ---")
provider_contact_query = "SELECT Name, Contact FROM providers WHERE City = 'New Jessica';"
provider_contact_df = pd.read_sql(provider_contact_query, engine)
print(provider_contact_df)


--- Contact Information for Providers in 'New Jessica' ---
               Name          Contact
0  Gonzales-Cochran  +1-600-220-0480


### 4. Which receivers have claimed the most food?

In [29]:

print("\n--- Top Receivers by Food Claimed ---")
top_receivers_query = """
    SELECT r.Name, SUM(fl.Quantity) AS TotalQuantityClaimed
    FROM receivers AS r
    JOIN claims AS c ON r.Receiver_ID = c.Receiver_ID
    JOIN food_listings AS fl ON c.Food_ID = fl.Food_ID
    GROUP BY r.Name
    ORDER BY TotalQuantityClaimed DESC
    LIMIT 10;
"""
top_receivers_df = pd.read_sql(top_receivers_query, engine)
print(top_receivers_df)


--- Top Receivers by Food Claimed ---
               Name  TotalQuantityClaimed
0      Matthew Webb                   191
1   Donald Caldwell                   174
2    Anthony Garcia                   162
3      Scott Hunter                   157
4   Jennifer Nelson                   139
5    Chelsea Powell                   134
6     Justin Powell                   126
7  Kristina Simpson                   124
8        Jose Vance                   123
9    Donna Williams                   122


### 5. What is the total quantity of food available from all providers?

In [30]:
print("\n--- Total Quantity of Food Available ---")
total_quantity_query = "SELECT SUM(Quantity) AS TotalAvailableFood FROM food_listings;"
total_quantity_df = pd.read_sql(total_quantity_query, engine)
print(total_quantity_df)


--- Total Quantity of Food Available ---
   TotalAvailableFood
0               25794


### 6. Which city has the highest number of food listings?

In [31]:

print("\n--- City with Highest Number of Food Listings ---")
highest_listings_query = "SELECT Location, COUNT(*) AS NumberOfListings FROM food_listings GROUP BY Location ORDER BY NumberOfListings DESC LIMIT 1;"
highest_listings_df = pd.read_sql(highest_listings_query, engine)
print(highest_listings_df)


--- City with Highest Number of Food Listings ---
        Location  NumberOfListings
0  South Kathryn                 6


### 7. What are the most commonly available food types?

In [32]:

print("\n--- Most Commonly Available Food Types ---")
most_common_food_type_query = "SELECT Food_Type, COUNT(*) AS NumberOfListings FROM food_listings GROUP BY Food_Type ORDER BY NumberOfListings DESC;"
most_common_food_type_df = pd.read_sql(most_common_food_type_query, engine)
print(most_common_food_type_df)


--- Most Commonly Available Food Types ---
        Food_Type  NumberOfListings
0      Vegetarian               336
1           Vegan               334
2  Non-Vegetarian               330


### 8. How many food claims have been made for each food item?

In [33]:

print("\n--- Claims Per Food Item ---")
claims_per_item_query = """
    SELECT fl.Food_Name, COUNT(c.Claim_ID) AS TotalClaims
    FROM food_listings AS fl
    JOIN claims AS c
    ON fl.Food_ID = c.Food_ID
    GROUP BY fl.Food_Name
    ORDER BY TotalClaims DESC;
"""
claims_per_item_df = pd.read_sql(claims_per_item_query, engine)
print(claims_per_item_df)


--- Claims Per Food Item ---
    Food_Name  TotalClaims
0        Rice          122
1        Soup          114
2       Dairy          110
3        Fish          108
4       Salad          106
5     Chicken          102
6       Bread           94
7       Pasta           87
8  Vegetables           86
9      Fruits           71


### 9. Which provider has had the highest number of successful food claims?

In [34]:

print("\n--- Provider with Most Successful Claims ---")
top_provider_successful_claims_query = """
    SELECT p.Name, COUNT(c.Claim_ID) AS SuccessfulClaims
    FROM providers AS p
    JOIN food_listings AS fl ON p.Provider_ID = fl.Provider_ID
    JOIN claims AS c ON fl.Food_ID = c.Food_ID
    WHERE c.Status = 'Completed'
    GROUP BY p.Name
    ORDER BY SuccessfulClaims DESC
    LIMIT 1;
"""
top_provider_successful_claims_df = pd.read_sql(top_provider_successful_claims_query, engine)
print(top_provider_successful_claims_df)


--- Provider with Most Successful Claims ---
          Name  SuccessfulClaims
0  Barry Group                 5


### 10. What percentage of food claims are completed vs. pending vs. canceled?

In [35]:

print("\n--- Percentage of Claims by Status ---")
claims_percentage_query = """
    SELECT
        Status,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
    FROM claims
    GROUP BY Status;
"""
claims_percentage_df = pd.read_sql(claims_percentage_query, engine)
print(claims_percentage_df)


--- Percentage of Claims by Status ---
      Status  Percentage
0  Cancelled        33.6
1  Completed        33.9
2    Pending        32.5


# 11. What is the average quantity of food claimed per receiver?

In [36]:

print("\n--- Average Quantity of Food Claimed Per Receiver ---")
avg_claimed_per_receiver_query = """
    SELECT AVG(T.TotalQuantity) AS AverageQuantityPerReceiver
    FROM (
        SELECT SUM(fl.Quantity) AS TotalQuantity
        FROM claims AS c
        JOIN food_listings AS fl ON c.Food_ID = fl.Food_ID
        GROUP BY c.Receiver_ID
    ) AS T;
"""
avg_claimed_per_receiver_df = pd.read_sql(avg_claimed_per_receiver_query, engine)
print(avg_claimed_per_receiver_df)


--- Average Quantity of Food Claimed Per Receiver ---
   AverageQuantityPerReceiver
0                   41.600962


### 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

In [37]:

print("\n--- Most Claimed Meal Type ---")
most_claimed_meal_type_query = """
    SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS NumberOfClaims
    FROM food_listings AS fl
    JOIN claims AS c ON fl.Food_ID = c.Food_ID
    GROUP BY fl.Meal_Type
    ORDER BY NumberOfClaims DESC
    LIMIT 1;
"""
most_claimed_meal_type_df = pd.read_sql(most_claimed_meal_type_query, engine)
print(most_claimed_meal_type_df)


--- Most Claimed Meal Type ---
   Meal_Type  NumberOfClaims
0  Breakfast             278


### 13. What is the total quantity of food donated by each provider?

In [38]:

print("\n--- Total Food Donated by Each Provider ---")
total_donated_by_provider_query = """
    SELECT p.Name, SUM(fl.Quantity) AS TotalDonatedQuantity
    FROM providers AS p
    JOIN food_listings AS fl
    ON p.Provider_ID = fl.Provider_ID
    GROUP BY p.Name
    ORDER BY TotalDonatedQuantity DESC;
"""
total_donated_by_provider_df = pd.read_sql(total_donated_by_provider_query, engine)
print(total_donated_by_provider_df.head(10))


--- Total Food Donated by Each Provider ---
                         Name  TotalDonatedQuantity
0                  Miller Inc                   217
1                 Barry Group                   179
2  Evans, Wright and Mitchell                   158
3                 Smith Group                   150
4                Campbell LLC                   145
5                  Nelson LLC                   142
6                  Ruiz-Oneal                   140
7           Blankenship-Lewis                   124
8                  Kelly-Ware                   123
9           Bradford-Martinez                   121


### 14. Top 5 cities with the highest number of food claims

In [39]:

print("\n--- Top 5 Cities with Highest Number of Claims ---")
top_cities_claims_query = """
    SELECT r.City, COUNT(c.Claim_ID) AS TotalClaims
    FROM claims AS c
    JOIN receivers AS r
    ON c.Receiver_ID = r.Receiver_ID
    GROUP BY r.City
    ORDER BY TotalClaims DESC
    LIMIT 5;
"""
top_cities_claims_df = pd.read_sql(top_cities_claims_query, engine)
print(top_cities_claims_df)



--- Top 5 Cities with Highest Number of Claims ---
           City  TotalClaims
0    West David            5
1    Smithshire            5
2  Port Richard            5
3     Port Dean            5
4      Greenton            5


### 15. Average quantity of food per listing, grouped by food type

In [40]:

print("\n--- Average Quantity Per Food Type ---")
avg_quantity_food_type_query = """
    SELECT Food_Type, AVG(Quantity) AS AverageQuantity
    FROM food_listings
    GROUP BY Food_Type;
"""
avg_quantity_food_type_df = pd.read_sql(avg_quantity_food_type_query, engine)
print(avg_quantity_food_type_df)


--- Average Quantity Per Food Type ---
        Food_Type  AverageQuantity
0  Non-Vegetarian        26.230303
1           Vegan        26.341317
2      Vegetarian        24.821429
